<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/tools/IsopropanolSynthesis_che2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che2.ipynb -O che2.ipynb
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/dotmap.ipynb -O dotmap.ipynb
!pip install importnb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from importnb import Notebook
with Notebook(): 
    from che2 import getLO, Range, RangeArray, Comp, CompArray, d2nt, Props
    from dotmap import DotMap

In [ ]:
import jax.numpy as jnp

<img src='https://github.com/profteachkids/CHE2064_Spring2022/raw/main/IsopropylSynthesisProcessDiagram.jpg'>

In [ ]:
# Static parameters (Total feed, feed mole fradtions, feed temperature and )
# 1. propene
# 2. diisopropyl ether
# 3. isopropanol
# 4. water
# Rx1:  C3H6 + H2O -> C3H8O
# Rx2:  2C3H8O -> C6H14 + H2O 

#Simple version - Overall mass balance only

e = DotMap()
e.alkeneF = 100.
e.waterF = 100.

e.alkeney = jnp.array([1., 0., 0., 0.])
e.waterx = jnp.array([0., 0., 0., 1.])

e.rx1 = jnp.array([-1, 0, 1, -1])
e.rx2 = jnp.array([0, 1, -2, 1])

e.rx1_extent = Range(e.alkeneF/2, 0., e.alkeneF)
e.rx2_extent = Range(e.alkeneF/4, 0., e.alkeneF/2)

e.alcoholP = Range(e.alkeneF/2, 0., e.alkeneF)  
e.alcoholPx = jnp.array([0., 0.005, 0.94, 0.055])

e.etherP = Range(e.alkeneF/2, 0., e.alkeneF)
e.etherPx = jnp.array([0.001, 0.98, 0.018, 0.002])

In [ ]:
x, x2nt, nt0  = d2nt(e)

In [ ]:
  def eqs(d):
    r=DotDict()
    feed = d.alkeneF*d.alkeney + d.waterF*d.waterx 
    product = (d.etherP*d.etherPx + d.alcoholP*d.alcoholPx)

    molecular_balance = feed + d.rx1_extent*d.rx1 + d.rx2_extent*d.rx2 - product

    r.alkene = d.alkeneF*d.alkeney
    r.water = d.waterF*d.waterx 
    r.ether = d.etherP*d.etherPx
    r.alcohol = d.alcoholP*d.alcoholPx
    return molecular_balance, r

In [ ]:
eqs_wrapped = wrap(eqs)
eqs_wrapped(x)

array([ 49.95, -24.25, -47.9 ,  72.15])

In [ ]:
sol=root(eqs_wrapped, x)
print(sol)
solx=sol.x

    fjac: array([[-6.20047158e-03,  6.42575319e-01,  7.65262557e-01,
         3.78382226e-02],
       [-8.63879645e-04,  6.72610397e-01, -5.87059363e-01,
         4.50506173e-01],
       [-3.25774691e-02,  3.66632454e-01, -2.64036850e-01,
        -8.91517747e-01],
       [-9.99449607e-01, -1.65183735e-02,  4.36621536e-03,
         2.84352445e-02]])
     fun: array([-1.40873424e-13, -1.84228632e-09, -2.05412221e-09, -1.19580790e-10])
 message: 'The solution converged.'
    nfev: 34
     qtf: array([-1.22865823e-07, -3.85024182e-09, -1.16130166e-09,  8.07790664e-10])
       r: array([ 0.92424769, -3.73988483, -2.98734898, -2.71847381, 10.08719343,
        1.92374923, -2.4314783 ,  1.11374846, -1.34568877,  0.04592532])
  status: 1
 success: True
       x: array([ 9.93202642, -2.14550313,  2.97450864, -2.97450864])


In [ ]:
dunk=xtodunk(solx)
dunk

{'alcoholP': array([95.14091351]),
 'etherP': array([4.85908649]),
 'rx1_extent': array([99.99514091]),
 'rx2_extent': array([5.23760933])}

In [ ]:
eqs_wrapped(sol.x)

array([-1.40873424e-13, -1.84228632e-09, -2.05412221e-09, -1.19580790e-10])

<img src='https://github.com/profteachkids/CHE2064_Spring2022/raw/main/IsopropylSynthesisProcessDiagram.jpg'>

In [ ]:
# Include internal flows (recycle)


d=DotDict()
d.alkeneF = 100.
d.waterF = 100.

d.alkeney = np.array([1., 0., 0., 0.])
d.waterx = np.array([0., 0., 0., 1.])

d.rx1 = np.array([-1, 0, 1, -1])
d.rx2 = np.array([0, 1, -2, 1])

d.rx1_extent = Range(d.alkeneF/2, 0., d.alkeneF)
d.rx2_extent = Range(d.alkeneF/4, 0., d.alkeneF/2)

d.alcoholP = Range(d.alkeneF/2, 0., d.alkeneF)  
d.alcoholPx = np.array([0., 0.005, 0.94, 0.055])

d.etherP = Range(d.alkeneF/2, 0., d.alkeneF)
d.etherPx = np.array([0.001, 0.98, 0.018, 0.002])

d.WARFR = 12. #water_alkene_reactor_feed_ratio

d.D1F = Range(d.alkeneF * d.WARFR, 0., 2.*d.WARFR*d.alkeneF)
d.D1Fx = Comp([0.001, 0.03, 0.07, 0.9])

d.D1K = np.array([100., 10., 5., 0.2])
d.D1Vy = Comp([0.01, 0.299, 0.3, 0.3])
d.D1Lx = Comp([0.0001, 0.1, 0.1, 0.8])
d.D1V = Range(d.alkeneF * d.WARFR/2, 0., 2.*d.WARFR*d.alkeneF)
d.D1L = Range(d.alkeneF * d.WARFR/2, 0., 2.*d.WARFR*d.alkeneF)

d.decantW = Range(d.alkeneF * d.WARFR, 0., 2.*d.WARFR*d.alkeneF)
d.decantWx = np.array([0.001, 0.005, 0.02, 0.98]) 


In [ ]:
wrap, x, d2, xtod, xtodunk  = dtox(d)

In [ ]:
def eqs(d):
    r=DotDict()
    feed = d.alkeneF*d.alkeney + d.waterF*d.waterx 
    product = (d.etherP*d.etherPx + d.alcoholP*d.alcoholPx)

    molecular_balance = feed + d.rx1_extent*d.rx1 + d.rx2_extent*d.rx2 - product

    r.alkene = d.alkeneF*d.alkeney
    r.water = d.waterF*d.waterx 
    r.ether = d.etherP*d.etherPx
    r.alcohol = d.alcoholP*d.alcoholPx
    return molecular_balance, r


In [ ]:
def eqs2(d):
    r=DotDict()
    r.reactor_feed = d.alkeneF*d.alkeney + d.waterF*d.waterx + d.D1L*d.D1Lx + d.decantW*d.decantWx

    r.REACTOR = r.reactor_feed + d.rx1_extent*d.rx1 + d.rx2_extent*d.rx2 - d.D1F*d.D1Fx

    r.FEED_RATIO = np.atleast_1d((d.waterF+d.D1L+d.decantW)/d.alkeneF - d.WARFR)
    
    r.D1 = d.D1F*d.D1Fx - d.D1V*d.D1Vy - d.D1L*d.D1Lx
    r.D1K = d.D1Vy/d.D1Lx - d.D1K

    r.AZ = d.D1V*d.D1Vy - d.etherP*d.etherPx - d.alcoholP*d.alcoholPx - d.decantW*d.decantWx

    r.alkene = d.alkeneF*d.alkeney
    r.water = d.waterF*d.waterx 
    r.ether = d.etherP*d.etherPx
    r.alcohol = d.alcoholP*d.alcoholPx
    return np.concatenate([r.REACTOR, r.FEED_RATIO, r.D1, r.D1K, r.AZ]), r


In [ ]:
eqs_wrapped = wrap(eqs2)

In [ ]:
eqs_wrapped(x)

array([ 5.00611928e+01,  5.50299646e+01,  7.79166839e-02,  6.52030926e+02,
        7.00000000e+00, -5.46185287e+00, -2.21389701e+02, -1.74097719e+02,
        4.00949272e+02,  1.00220022e+01, -6.71034213e+00, -1.69933993e+00,
        2.12582508e-01,  5.35066007e+00,  1.42109736e+02,  1.26119802e+02,
       -9.80830198e+02])

In [ ]:
sol=root(eqs_wrapped, x)
print(sol)
solx=sol.x

In [ ]:
dunk=xtodunk(solx)
dunk

{'D1F': array([1200.09006814]),
 'D1Fx': array([1.73915272e-05, 8.62596534e-03, 2.17013664e-01, 7.74342979e-01]),
 'D1L': array([1085.79849147]),
 'D1Lx': array([1.66771652e-06, 4.64480150e-03, 1.57148856e-01, 8.38204675e-01]),
 'D1V': array([114.29157667]),
 'D1Vy': array([1.66771652e-04, 4.64480150e-02, 7.85744278e-01, 1.67640935e-01]),
 'alcoholP': array([95.14091351]),
 'decantW': array([14.20150853]),
 'etherP': array([4.85908649]),
 'rx1_extent': array([99.99514091]),
 'rx2_extent': array([5.23760933])}

In [ ]:
eqs_wrapped(sol.x)

array([-2.62082023e-14,  0.00000000e+00,  5.68434189e-14,  2.27373675e-13,
        0.00000000e+00,  9.87491339e-16,  0.00000000e+00, -2.84217094e-14,
       -1.13686838e-13, -5.68434189e-14,  1.77635684e-15,  0.00000000e+00,
        2.22044605e-16, -8.96852037e-16,  1.38777878e-15, -2.44249065e-15,
        2.13162821e-14])